<a href="https://colab.research.google.com/github/murakami-tatsumi/colab/blob/master/BERT%E3%81%AE%E3%83%95%E3%82%A1%E3%82%A4%E3%83%B3%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E4%BA%8B%E4%BE%8B%EF%BC%88%E6%97%A5%E6%9C%AC%E8%AA%9Esentense_piece%EF%BC%89livedoor%E8%A8%98%E4%BA%8B%E3%81%AE%E5%88%86%E9%A1%9E%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Finetuning of the pretrained Japanese BERT model

yoheikikutaさんの下記githubタスクを再現<br>
https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb

Finetune the pretrained model to solve multi-class classification problems.  
This notebook requires the following objects:
- trained sentencepiece model (model and vocab files)
- pretraiend Japanese BERT model

Dataset is livedoor ニュースコーパス in https://www.rondhuit.com/download.html.  
We make test:dev:train = 2:2:6 datasets.

Results:

- Full training data
  - BERT with SentencePiece
    ```
                    precision    recall  f1-score   support

    dokujo-tsushin       0.98      0.94      0.96       178
      it-life-hack       0.96      0.97      0.96       172
     kaden-channel       0.99      0.98      0.99       176
    livedoor-homme       0.98      0.88      0.93        95
       movie-enter       0.96      0.99      0.98       158
            peachy       0.94      0.98      0.96       174
              smax       0.98      0.99      0.99       167
      sports-watch       0.98      1.00      0.99       190
        topic-news       0.99      0.98      0.98       163

         micro avg       0.97      0.97      0.97      1473
         macro avg       0.97      0.97      0.97      1473
      weighted avg       0.97      0.97      0.97      1473
    ```
  - sklearn GradientBoostingClassifier with MeCab
    ```
                      precision    recall  f1-score   support

    dokujo-tsushin       0.89      0.86      0.88       178
      it-life-hack       0.91      0.90      0.91       172
     kaden-channel       0.90      0.94      0.92       176
    livedoor-homme       0.79      0.74      0.76        95
       movie-enter       0.93      0.96      0.95       158
            peachy       0.87      0.92      0.89       174
              smax       0.99      1.00      1.00       167
      sports-watch       0.93      0.98      0.96       190
        topic-news       0.96      0.86      0.91       163

         micro avg       0.92      0.92      0.92      1473
         macro avg       0.91      0.91      0.91      1473
      weighted avg       0.92      0.92      0.91      1473
    ```

- Small training data (1/5 of full training data)
  - BERT with SentencePiece
    ```
                    precision    recall  f1-score   support

    dokujo-tsushin       0.97      0.87      0.92       178
      it-life-hack       0.86      0.86      0.86       172
     kaden-channel       0.95      0.94      0.95       176
    livedoor-homme       0.82      0.82      0.82        95
       movie-enter       0.97      0.99      0.98       158
            peachy       0.89      0.95      0.92       174
              smax       0.94      0.96      0.95       167
      sports-watch       0.97      0.97      0.97       190
        topic-news       0.94      0.94      0.94       163

         micro avg       0.93      0.93      0.93      1473
         macro avg       0.92      0.92      0.92      1473
      weighted avg       0.93      0.93      0.93      1473
    ```
  - sklearn GradientBoostingClassifier with MeCab
    ```
                    precision    recall  f1-score   support

    dokujo-tsushin       0.82      0.71      0.76       178
      it-life-hack       0.86      0.88      0.87       172
     kaden-channel       0.91      0.87      0.89       176
    livedoor-homme       0.67      0.63      0.65        95
       movie-enter       0.87      0.95      0.91       158
            peachy       0.70      0.78      0.73       174
              smax       1.00      1.00      1.00       167
      sports-watch       0.87      0.95      0.91       190
        topic-news       0.92      0.82      0.87       163

         micro avg       0.85      0.85      0.85      1473
         macro avg       0.85      0.84      0.84      1473
      weighted avg       0.86      0.85      0.85      1473
    ```

## オリジナルに対する追加設定

下の手順で環境を自分のcolabに構築します。

下の手順以外に次の設定を前提としています。

- ランタイムはTPUを使用　→TPUでは時間がかかり過ぎたのでGPUに変更
- マイドライブにyoheikikutaさんのmodelを追加しておく

→yoheikikutaさんのgoogle drive
> https://drive.google.com/drive/folders/1Zsm9DD40lrUVu6iAnIuTH2ODIkh-WM-O<br>

をgoogle driveにて「マイドライブ」に追加しておく<br>
（マイドライブ経由で直接アクセスもできるが、コピーも早いので安全策をとる）


In [1]:
# google driveへの接続は毎回必要
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [21]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 39.5MB/s 


In [2]:
# bert-japaneseをgoogle driveのコード用フォルダにロード
!git clone https://github.com/yoheikikuta/bert-japanese.git

Cloning into 'bert-japanese'...
remote: Enumerating objects: 144, done.
remote: Total 144 (delta 0), reused 0 (delta 0), pack-reused 144
Receiving objects: 100% (144/144), 208.67 KiB | 436.00 KiB/s, done.
Resolving deltas: 100% (78/78), done.


In [3]:
%cd /content/bert-japanese
!ls -l
!ls -l src

/content/bert-japanese
total 208
drwxr-xr-x 2 root root   4096 Sep  7 00:19 bert
-rw-r--r-- 1 root root    789 Sep  7 00:19 config.ini
drwxr-xr-x 3 root root   4096 Sep  7 00:19 data
-rw-r--r-- 1 root root    726 Sep  7 00:19 Dockerfile
-rw-r--r-- 1 root root  11357 Sep  7 00:19 LICENSE
drwxr-xr-x 2 root root   4096 Sep  7 00:19 model
drwxr-xr-x 2 root root   4096 Sep  7 00:19 notebook
-rw-r--r-- 1 root root 158420 Sep  7 00:19 pretraining-loss.png
-rw-r--r-- 1 root root   5539 Sep  7 00:19 README.md
-rw-r--r-- 1 root root     40 Sep  7 00:19 requirements.txt
drwxr-xr-x 2 root root   4096 Sep  7 00:19 src
total 108
-rw-r--r-- 1 root root 15049 Sep  7 00:19 create_pretraining_data.py
-rw-r--r-- 1 root root  1336 Sep  7 00:19 data-download-and-extract.py
-rw-r--r-- 1 root root 14285 Sep  7 00:19 extract_features.py
-rw-r--r-- 1 root root  1168 Sep  7 00:19 file-preprocessing.sh
-rw-r--r-- 1 root root 30778 Sep  7 00:19 run_classifier.py
-rw-r--r-- 1 root root 18694 Sep  7 00:19 run_pretr

bert-japaneseの下のフォルダ構成は
> bert-japanese
> - bert　　　→ フォルダのみ
> - model　　 → フォルダのみ
> - notebook　→ 処理には不要
> - src

のようになっている。
- bert　　→本家のbertをgitで展開
- model   →yoheikikutaさんのgoogle driveよりコピー


In [4]:
# bertをbertフォルダにロード
!git clone https://github.com/google-research/bert.git
!ls -l bert
import sys
sys.path.append('/content/bert-japanese/bert')
sys.path

Cloning into 'bert'...
remote: Enumerating objects: 333, done.
remote: Total 333 (delta 0), reused 0 (delta 0), pack-reused 333
Receiving objects: 100% (333/333), 282.45 KiB | 559.00 KiB/s, done.
Resolving deltas: 100% (183/183), done.
total 396
-rw-r--r-- 1 root root  1323 Sep  7 00:20 CONTRIBUTING.md
-rw-r--r-- 1 root root 16475 Sep  7 00:20 create_pretraining_data.py
-rw-r--r-- 1 root root 13898 Sep  7 00:20 extract_features.py
-rw-r--r-- 1 root root   616 Sep  7 00:20 __init__.py
-rw-r--r-- 1 root root 11358 Sep  7 00:20 LICENSE
-rw-r--r-- 1 root root 37922 Sep  7 00:20 modeling.py
-rw-r--r-- 1 root root  9191 Sep  7 00:20 modeling_test.py
-rw-r--r-- 1 root root 11220 Sep  7 00:20 multilingual.md
-rw-r--r-- 1 root root  6258 Sep  7 00:20 optimization.py
-rw-r--r-- 1 root root  1721 Sep  7 00:20 optimization_test.py
-rw-r--r-- 1 root root 66488 Sep  7 00:20 predicting_movie_reviews_with_bert_on_tf_hub.ipynb
-rw-r--r-- 1 root root 45390 Sep  7 00:20 README.md
-rw-r--r-- 1 root root  

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/bert-japanese/bert']

In [5]:
# yoheikikutaさんのgoogle driveよりmodelファイルをコピー
# それなり（10分程度）の時間はかかる
%%time
!cp /content/drive/My\ Drive/bert-wiki-ja/*.* ./model/
!ls -l model

total 4158500
-r-------- 1 root root   11026495 Sep  7 00:20 graph.pbtxt
-r-------- 1 root root 2906079739 Sep  7 00:21 jawiki-20181220-pages-articles-multistream.xml.bz2
-r-------- 1 root root 1334916128 Sep  7 00:21 model.ckpt-1400000.data-00000-of-00001
-r-------- 1 root root       9349 Sep  7 00:21 model.ckpt-1400000.index
-r-------- 1 root root    4843824 Sep  7 00:21 model.ckpt-1400000.meta
-r-------- 1 root root     805634 Sep  7 00:21 wiki-ja.model
-r-------- 1 root root     595681 Sep  7 00:21 wiki-ja.vocab
CPU times: user 381 ms, sys: 37.1 ms, total: 418 ms
Wall time: 1min 7s


オリジナルのconfig.iniはcolab環境のフォルダ構成を
> /work/

直下にbert-japaneseが展開された想定になっているので、
> /work/　→　/content/bert-japanese/

に置き換えたconfig.iniに置換する必要がある

In [6]:
# 修正したconfig.iniをアップロード
!rm config.ini
from google.colab import files
uploaded = files.upload()

Saving config.ini to config.ini


In [7]:
# カレントフォルダをsrcに移動
%cd /content/bert-japanese/src

/content/bert-japanese/src


## ここからいよいよ実行

In [8]:
import configparser
import glob
import os
import pandas as pd
import subprocess
import sys
import tarfile 
from urllib.request import urlretrieve

CURDIR = os.getcwd()
CONFIGPATH = os.path.join(CURDIR, os.pardir, 'config.ini')
config = configparser.ConfigParser()
config.read(CONFIGPATH)

['/content/bert-japanese/src/../config.ini']

## Data preparing （→題材であるlivedoor記事をFine Tuningの入力用に加工する）

You need execute the following cells just once.

In [0]:
FILEURL = config['FINETUNING-DATA']['FILEURL']
FILEPATH = config['FINETUNING-DATA']['FILEPATH']
EXTRACTDIR = config['FINETUNING-DATA']['TEXTDIR']

Download and unzip data.

In [10]:
%%time

urlretrieve(FILEURL, FILEPATH)

mode = "r:gz"
tar = tarfile.open(FILEPATH, mode) 
tar.extractall(EXTRACTDIR) 
tar.close()

CPU times: user 1.12 s, sys: 525 ms, total: 1.64 s
Wall time: 1.86 s


Data preprocessing.

In [0]:
def extract_txt(filename):
    with open(filename) as text_file:
        # 0: URL, 1: timestamp
        text = text_file.readlines()[2:]
        text = [sentence.strip() for sentence in text]
        text = list(filter(lambda line: line != '', text))
        return ''.join(text)

In [0]:
categories = [ 
    name for name 
    in os.listdir( os.path.join(EXTRACTDIR, "text") ) 
    if os.path.isdir( os.path.join(EXTRACTDIR, "text", name) ) ]

categories = sorted(categories)

In [13]:
categories

['dokujo-tsushin',
 'it-life-hack',
 'kaden-channel',
 'livedoor-homme',
 'movie-enter',
 'peachy',
 'smax',
 'sports-watch',
 'topic-news']

In [0]:
table = str.maketrans({
    '\n': '',
    '\t': '　',
    '\r': '',
})

In [15]:
%%time

all_text = []
all_label = []

for cat in categories:
    files = glob.glob(os.path.join(EXTRACTDIR, "text", cat, "{}*.txt".format(cat)))
    files = sorted(files)
    body = [ extract_txt(elem).translate(table) for elem in files ]
    label = [cat] * len(body)
    
    all_text.extend(body)
    all_label.extend(label)

CPU times: user 871 ms, sys: 97 ms, total: 968 ms
Wall time: 976 ms


In [0]:
df = pd.DataFrame({'text' : all_text, 'label' : all_label})

In [17]:
df.head()

,text,label
0,友人代表のスピーチ、独女はどうこなしている？もうすぐジューン・ブライドと呼ばれる６月。独女の...,dokujo-tsushin
1,ネットで断ち切れない元カレとの縁携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だ...,dokujo-tsushin
2,相次ぐ芸能人の“すっぴん”披露 その時、独女の心境は？「男性はやっぱり、女性の“すっぴん”が...,dokujo-tsushin
3,ムダな抵抗！？ 加齢の現実ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「...,dokujo-tsushin
4,税金を払うのは私たちなんですけど！6月から支給される子ども手当だが、当初は子ども一人当たり月...,dokujo-tsushin


In [0]:
df = df.sample(frac=1, random_state=23).reset_index(drop=True)

In [19]:
df.head()

,text,label
0,新記録でロンドンに乗り込む“バタフライの女王”加藤ゆか3日に行われた競泳の日本選手権で、女子...,sports-watch
1,家電チャンネルの記事も配信！向かうところ敵なしのスマホアプリ「ITニュース by lived...,kaden-channel
2,彼にあげたい韓国メンズコスメ、韓流俳優のような美肌男へ！年末の大イベント、クリスマスまであと...,peachy
3,快適なスマホライフのための必須アプリ「マトリックス レボリューションズ」(c)Warner ...,livedoor-homme
4,独女と上司の気になる関係人事異動の多い春は、職場の人間関係の悩みも増える時期。『an・an』...,dokujo-tsushin


Save data as tsv files.  
test:dev:train = 2:2:6. To check the usability of finetuning, we also prepare sampled training data (1/5 of full training data).

In [0]:
df[:len(df) // 5].to_csv( os.path.join(EXTRACTDIR, "test.tsv"), sep='\t', index=False)
df[len(df) // 5:len(df)*2 // 5].to_csv( os.path.join(EXTRACTDIR, "dev.tsv"), sep='\t', index=False)
df[len(df)*2 // 5:].to_csv( os.path.join(EXTRACTDIR, "train.tsv"), sep='\t', index=False)

### 1/5 of full training data.
# df[:len(df) // 5].to_csv( os.path.join(EXTRACTDIR, "test.tsv"), sep='\t', index=False)
# df[len(df) // 5:len(df)*2 // 5].to_csv( os.path.join(EXTRACTDIR, "dev.tsv"), sep='\t', index=False)
# df[len(df)*2 // 5:].sample(frac=0.2, random_state=23).to_csv( os.path.join(EXTRACTDIR, "train.tsv"), sep='\t', index=False)

In [0]:
PRETRAINED_MODEL_PATH = '../model/model.ckpt-1400000'
FINETUNE_OUTPUT_DIR = '../model/livedoor_output'

## Finetune pre-trained model

It will take a lot of hours to execute the following cells on CPU environment.  
You can also use colab to recieve the power of TPU. You need to uplode the created data onto your GCS bucket.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1zZH2GWe0U-7GjJ2w2duodFfEUptvHjcx)

In [0]:
%%time
# It will take many hours on CPU environment.

!python3 ../src/run_classifier.py \
  --task_name=livedoor \
  --do_train=true \
  --do_eval=true \
  --data_dir=../data/livedoor \
  --model_file="../model/wiki-ja.model" \
  --vocab_file="../model/wiki-ja.vocab" \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --max_seq_length=512 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=10 \
  --output_dir={FINETUNE_OUTPUT_DIR}

W0902 23:59:58.584618 139745888335744 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0902 23:59:58.585610 139745888335744 deprecation_wrapper.py:119] From ../src/run_classifier.py:854: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0902 23:59:58.586091 139745888335744 deprecation_wrapper.py:119] From ../src/run_classifier.py:659: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0902 23:59:58.586206 139745888335744 deprecation_wrapper.py:119] From ../src/run_classifier.py:659: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0902 23:59:58.586540 139745888335744 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0902 23:

In [0]:
!ls -l ../model/livedoor_output
!ls -l ../model/livedoor_output/eval

total 6538164
-rw-r--r-- 1 root root        275 Sep  3 01:53 checkpoint
drwxr-xr-x 2 root root       4096 Sep  3 01:55 eval
-rw-r--r-- 1 root root         87 Sep  3 01:55 eval_results.txt
-rw-r--r-- 1 root root    2960154 Sep  3 01:53 eval.tf_record
-rw-r--r-- 1 root root   15616531 Sep  3 01:53 events.out.tfevents.1567468817.ed6a29adb36d
-rw-r--r-- 1 root root    9634556 Sep  3 00:00 graph.pbtxt
-rw-r--r-- 1 root root 1327491188 Sep  3 01:43 model.ckpt-10000.data-00000-of-00001
-rw-r--r-- 1 root root      22767 Sep  3 01:43 model.ckpt-10000.index
-rw-r--r-- 1 root root    4071018 Sep  3 01:43 model.ckpt-10000.meta
-rw-r--r-- 1 root root 1327491188 Sep  3 01:53 model.ckpt-11000.data-00000-of-00001
-rw-r--r-- 1 root root      22767 Sep  3 01:53 model.ckpt-11000.index
-rw-r--r-- 1 root root    4071018 Sep  3 01:53 model.ckpt-11000.meta
-rw-r--r-- 1 root root 1327491188 Sep  3 01:53 model.ckpt-11052.data-00000-of-00001
-rw-r--r-- 1 root root      22767 Sep  3 01:53 model.ckpt-11052.index


In [0]:
# Fine Tuning結果をMy Driveに保存（下記はコピー先修正済み。最初に実行した時はコピー先を間違えて下記より1段深くなっている）
!cp -r ../model/livedoor_output /content/drive/My\ Drive/Colab_Data/

## Predict using the finetuned model

Let's predict test data using the finetuned model.  

In [0]:
# Fine Tuning結果をMy Driveから復元（最初のコピー先を間違えたので1段深くなっている）
!cp -r /content/drive/My\ Drive/Colab_Data/livedoor_output/livedoor_output ../model/

In [24]:
!ls -l ../model/livedoor_output

total 6538164
-rw------- 1 root root        275 Sep  7 00:27 checkpoint
drwx------ 2 root root       4096 Sep  7 00:27 eval
-rw------- 1 root root         87 Sep  7 00:28 eval_results.txt
-rw------- 1 root root    2960154 Sep  7 00:27 eval.tf_record
-rw------- 1 root root   15616531 Sep  7 00:27 events.out.tfevents.1567468817.ed6a29adb36d
-rw------- 1 root root    9634556 Sep  7 00:27 graph.pbtxt
-rw------- 1 root root 1327491188 Sep  7 00:28 model.ckpt-10000.data-00000-of-00001
-rw------- 1 root root      22767 Sep  7 00:28 model.ckpt-10000.index
-rw------- 1 root root    4071018 Sep  7 00:27 model.ckpt-10000.meta
-rw------- 1 root root 1327491188 Sep  7 00:28 model.ckpt-11000.data-00000-of-00001
-rw------- 1 root root      22767 Sep  7 00:28 model.ckpt-11000.index
-rw------- 1 root root    4071018 Sep  7 00:27 model.ckpt-11000.meta
-rw------- 1 root root 1327491188 Sep  7 00:27 model.ckpt-11052.data-00000-of-00001
-rw------- 1 root root      22767 Sep  7 00:28 model.ckpt-11052.index


In [26]:
import sys
sys.path.append("../src")

import tokenization_sentencepiece as tokenization
from run_classifier import LivedoorProcessor
from run_classifier import model_fn_builder
from run_classifier import file_based_input_fn_builder
from run_classifier import file_based_convert_examples_to_features
from utils import str_to_value

In [0]:
sys.path.append("../bert")

import modeling
import optimization
import tensorflow as tf

In [28]:
import configparser
import json
import glob
import os
import pandas as pd
import tempfile

bert_config_file = tempfile.NamedTemporaryFile(mode='w+t', encoding='utf-8', suffix='.json')
bert_config_file.write(json.dumps({k:str_to_value(v) for k,v in config['BERT-CONFIG'].items()}))
bert_config_file.seek(0)
bert_config = modeling.BertConfig.from_json_file(bert_config_file.name)

In [0]:
# 以下の式は問題あり。文字列をSORTしているので数字の桁上がりの考慮がないためlatestになっていない
# フォルダ内を確認して、場合によっては FINETUNED_MODEL_PATH を手で修正する
output_ckpts = glob.glob("{}/model.ckpt*data*".format(FINETUNE_OUTPUT_DIR))
latest_ckpt = sorted(output_ckpts)[-1]
FINETUNED_MODEL_PATH = latest_ckpt.split('.data-00000-of-00001')[0]

In [0]:
class FLAGS(object):
    '''Parameters.'''
    def __init__(self):
        self.model_file = "../model/wiki-ja.model"
        self.vocab_file = "../model/wiki-ja.vocab"
        self.do_lower_case = True
        self.use_tpu = False
        self.output_dir = "/dummy"
        self.data_dir = "../data/livedoor"
        self.max_seq_length = 512
        self.init_checkpoint = FINETUNED_MODEL_PATH
        self.predict_batch_size = 4
        
        # The following parameters are not used in predictions.
        # Just use to create RunConfig.
        self.master = None
        self.save_checkpoints_steps = 1
        self.iterations_per_loop = 1
        self.num_tpu_cores = 1
        self.learning_rate = 0
        self.num_warmup_steps = 0
        self.num_train_steps = 0
        self.train_batch_size = 0
        self.eval_batch_size = 0

In [0]:
FLAGS = FLAGS()

In [0]:
processor = LivedoorProcessor()
label_list = processor.get_labels()

In [39]:
tokenizer = tokenization.FullTokenizer(
    model_file=FLAGS.model_file, vocab_file=FLAGS.vocab_file,
    do_lower_case=FLAGS.do_lower_case)

tpu_cluster_resolver = None

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

Loaded a trained SentencePiece model.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [40]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=FLAGS.num_train_steps,
    num_warmup_steps=FLAGS.num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_tpu)


estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

INFO:tensorflow:Using config: {'_model_dir': '/dummy', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f901f6d1320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1, num_shards=1, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initi

In [41]:
predict_examples = processor.get_test_examples(FLAGS.data_dir)
predict_file = tempfile.NamedTemporaryFile(mode='w+t', encoding='utf-8', suffix='.tf_record')

file_based_convert_examples_to_features(predict_examples, label_list,
                                        FLAGS.max_seq_length, tokenizer,
                                        predict_file.name)

predict_drop_remainder = True if FLAGS.use_tpu else False

predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file.name,
    seq_length=FLAGS.max_seq_length,
    is_training=False,
    drop_remainder=predict_drop_remainder)




INFO:tensorflow:Writing example 0 of 1473
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-1
INFO:tensorflow:tokens: [CLS] ▁ 新記録 で ロンドン に乗り 込む “ バタ フライ の 女王 ” 加藤 ゆ か 3 日に行われた 競泳 の 日本選手権 で 、 女子 100 メートル バタ フライ の 加藤 ゆ か ( 25 歳 ) が 2 大会連続 の 五輪 出場 を決めた 。 57 秒 77 と 、 自身が 持つ 日本 新 記録を更新 して の 五輪 切符 ゲット だ 。 前回 大会の 北京 五輪 選考 では ガ チ ガ チ に 緊張 していたという 加藤 は 、 幾 多 の経験 を得て 、 強い 精神 力を 培 った 。 記録 更新 での 五輪 出場権 獲得 に 、 爽 やかな 笑顔 で 喜び を 爆発 させた 。 百 花 繚 乱 の 日本女子 競泳 界 の中でも 、 加藤 は 美女 スイ マー の 筆頭 に数えられる 。 目 鼻 立ち の 整 った 顔 に 、 白く 透 き 通 るような 肌 、 そして 鍛え あげられ た アスリート の 肉体 に 、 ネット 上で の人気 も 上 々 だ 。 前回 の 北京 五輪 では 予選 敗退 で 悔 し 涙 を 呑 んだ 加藤 。 4 年間の 濃 密 な 時間 を経て 、 速く 、 より 美しく なった 「 バタ フライ の 女王 」 が 、 ロンドンの プール を 沸 かす 。 ・ 加藤 ゆ か ▁ フォト [SEP]
INFO:tensorflow:input_ids: 4 9 19861 19 1418 4641 2161 1314 16012 5699 10 4685 809 3995 2234 95 31 4533 30140 10 22440 19 7 712 431 1346 16012 5699 10 3995 2234 95 15 228 559 14 12 25 20703 10 7060 840 6210 8 2487 1053 3168 20 7 3310 7506 99 123 20736 55 10 7060 24989 26563 314 8 9

In [0]:
result = estimator.predict(input_fn=predict_input_fn)

In [43]:
%%time
# It will take a few hours on CPU environment.

result = list(result)

INFO:tensorflow:Could not find trained model in model_dir: /dummy, running initialization to predict.
Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.

Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 512)
INFO:tensorflow:  name = input_mask, shape = (?, 512)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 512)



Instructions for updating:
Use keras.layers.dense instead.

INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape =

In [46]:
result[:2]

[{'probabilities': array([2.0330683e-06, 2.5587556e-06, 3.4543314e-06, 1.7470909e-06,
         3.0898557e-06, 2.5465058e-06, 3.1702596e-06, 9.9997365e-01,
         7.7879604e-06], dtype=float32)},
 {'probabilities': array([4.7826766e-06, 9.9372110e-06, 9.9995136e-01, 7.0410892e-06,
         5.5301525e-06, 3.4652683e-06, 6.4880032e-06, 5.7532570e-06,
         5.7741827e-06], dtype=float32)}]

Read test data set and add prediction results.

In [0]:
import pandas as pd

In [0]:
test_df = pd.read_csv("../data/livedoor/test.tsv", sep='\t')

In [0]:
test_df['predict'] = [ label_list[elem['probabilities'].argmax()] for elem in result ]

In [50]:
test_df.head()

,text,label,predict
0,新記録でロンドンに乗り込む“バタフライの女王”加藤ゆか3日に行われた競泳の日本選手権で、女子...,sports-watch,sports-watch
1,家電チャンネルの記事も配信！向かうところ敵なしのスマホアプリ「ITニュース by lived...,kaden-channel,kaden-channel
2,彼にあげたい韓国メンズコスメ、韓流俳優のような美肌男へ！年末の大イベント、クリスマスまであと...,peachy,peachy
3,快適なスマホライフのための必須アプリ「マトリックス レボリューションズ」(c)Warner ...,livedoor-homme,livedoor-homme
4,独女と上司の気になる関係人事異動の多い春は、職場の人間関係の悩みも増える時期。『an・an』...,dokujo-tsushin,dokujo-tsushin


In [51]:
sum( test_df['label'] == test_df['predict'] ) / len(test_df)

0.9660556687033266

A littel more detailed check using `sklearn.metrics`.

In [52]:
!pip install scikit-learn

In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [54]:
print(classification_report(test_df['label'], test_df['predict']))

                precision    recall  f1-score   support

dokujo-tsushin       0.99      0.92      0.95       178
  it-life-hack       0.95      0.94      0.95       172
 kaden-channel       0.99      0.98      0.99       176
livedoor-homme       0.98      0.88      0.93        95
   movie-enter       0.97      0.99      0.98       158
        peachy       0.93      0.99      0.96       174
          smax       0.94      1.00      0.97       167
  sports-watch       0.98      0.98      0.98       190
    topic-news       0.96      0.98      0.97       163

      accuracy                           0.97      1473
     macro avg       0.97      0.96      0.96      1473
  weighted avg       0.97      0.97      0.97      1473



In [55]:
print(confusion_matrix(test_df['label'], test_df['predict']))

[[164   1   1   1   2   7   0   0   2]
 [  0 162   0   1   0   1   8   0   0]
 [  0   3 172   0   0   1   0   0   0]
 [  1   3   0  84   3   2   0   0   2]
 [  0   1   0   0 156   1   0   0   0]
 [  1   0   0   0   0 172   1   0   0]
 [  0   0   0   0   0   0 167   0   0]
 [  0   0   0   0   0   0   0 187   3]
 [  0   0   0   0   0   0   1   3 159]]


### Simple baseline model.

In [0]:
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [0]:
train_df = pd.read_csv("../data/livedoor/train.tsv", sep='\t')
dev_df = pd.read_csv("../data/livedoor/dev.tsv", sep='\t')
test_df = pd.read_csv("../data/livedoor/test.tsv", sep='\t')

In [0]:
!apt-get install -q -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

In [0]:
!pip install mecab-python3==0.7

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
import MeCab

In [0]:
m = MeCab.Tagger("-Owakati")

In [0]:
train_dev_df = pd.concat([train_df, dev_df])

In [0]:
train_dev_xs = train_dev_df['text'].apply(lambda x: m.parse(x))
train_dev_ys = train_dev_df['label']

test_xs = test_df['text'].apply(lambda x: m.parse(x))
test_ys = test_df['label']

In [0]:
vectorizer = TfidfVectorizer(max_features=750)
train_dev_xs_ = vectorizer.fit_transform(train_dev_xs)
test_xs_ = vectorizer.transform(test_xs)

The following set up is not exactly identical to that of BERT because inside Classifier it uses `train_test_split` with shuffle.  
In addition, parameters are not well tuned, however, we think it's enough to check the power of BERT.

In [0]:
%%time

model = GradientBoostingClassifier(n_estimators=200,
                                   validation_fraction=len(dev_df)/len(train_df),
                                   n_iter_no_change=5,
                                   tol=0.01,
                                   random_state=23)

### 1/5 of full training data.
# model = GradientBoostingClassifier(n_estimators=200,
#                                    validation_fraction=len(dev_df)/len(train_df),
#                                    n_iter_no_change=5,
#                                    tol=0.01,
#                                    random_state=23)

model.fit(train_dev_xs_, train_dev_ys)

In [0]:
print(classification_report(test_ys, model.predict(test_xs_)))

In [0]:
print(confusion_matrix(test_ys, model.predict(test_xs_)))